In [1]:
from elasticsearch import (
    Elasticsearch,
    helpers,
)  # Import the Elasticsearch client and helpers module
client = Elasticsearch('http://172.31.0.2:9200')
client.ping()


True

In [12]:
!pip3 -q install eland sentence_transformers transformers 


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
onnx-graphsurgeon 0.3.12 requires onnx, which is not installed.


## Using sematic search

In [4]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

/home/edtechai/venv/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [25]:
client.indices.delete(index='ecommerce_search')

ObjectApiResponse({'acknowledged': True})

In [16]:
client.indices.create(index='test', timeout='5s')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'test'})

In [39]:
mapping = {
    "properties": {
        "id": {
            "type": "text",
        },
        "description": {
            "type": "text",
        },
        "category": {
            "type": "text",
        },
        "title": {
            "type": "text",
        },
        "brand": {
            "type": "text",
        },
        "mrp": {
            "type": "text",
        },
        "price": {
            "type": "text",
        },
        "offers": {
            "type": "text",
        },
        "stock_availibility": {
            "type": "boolean",
        },
        "product_asin": {
            "type": "text",
            "fields": {"keyword": {"type": "keyword", "ignore_above": 50}},
        },
        "image_urls": {
            "type": "text",
        },
        "description_vector": {  # Inference results field, target_field.predicted_value
            "type": "dense_vector",
            "dims": 768,  # The all-mpnet-base-v2 model has embedding_size of 768, so dims is set to 768.
            "index": "true",
            "similarity": "l2_norm",  #  When indexing vectors for approximate kNN search, you need to specify the similarity function for comparing the vectors.
        },
    },
}
client.indices.create(index="ecommerce_search", mappings=mapping, timeout='1s')

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': False, 'index': 'ecommerce_search'})

In [2]:
import pandas as pd
df = pd.read_csv('./clean_products.csv')
df.head()

,id,category,title,description,brand,mrp,price,offers,stock_availibility,product_asin,image_urls
0,eb49cc038190f6f03c272f79fbbce894,Skin Care,Lee posh Lactic Acid 60% Anti ageing Pigmenta...,PROFESSIONAL GRADE Face Peel: this peel stimul...,Lee Posh,2000.00,799.00,60.05%,True,B072BGHNJ1,https://images-na.ssl-images-amazon.com/images...
1,1657cc30c438affede6a5060d6847363,Skin Care,Branded SLB Works New 1.5mm Titanium 1200 nee...,Item name: 1.5mm titanium 1200 needles microne...,SLB Works,2040.00,2040.00,0%,True,B07QDTZYSJ,https://images-na.ssl-images-amazon.com/images...
2,41654633cce38c8650690f6dbac01fd3,Skin Care,Generic 1 Pc brand snail eye cream remove dar...,"Use: eye, item type: cream, net wt: 20g, gzzz:...",Generic,1824.00,1042.00,42.87%,True,B07DCSN8MP,https://images-na.ssl-images-amazon.com/images...
3,08b1bd85c3efc2d7aa556fd79b073382,Skin Care,Generic Anti Snoring Snore Stopper Sleep Apne...,Prevent the tongue from dropping backward or b...,Generic,2185.00,1399.00,35.97%,True,B07GLW9VQN,https://images-na.ssl-images-amazon.com/images...
4,3ac3f213732512d1d11bb73ab3b1900f,Grocery & Gourmet Foods,Harveys Crunchy & Creame Gourmet Delicacies C...,Harvey's wafer Cream Wafer 110g. Made in India,Harveys,594.00,570.00,4.04%,True,B07NFYYLF1,https://images-na.ssl-images-amazon.com/images...


In [3]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-mpnet-base-v2')

c:\Users\lengu\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
df['title_description'] = 'Product name is ' + df['title'] +'. With description: '+ df['description']

In [5]:
df['title_description']

0        Product name is  Lee posh Lactic Acid 60% Anti...
1        Product name is  Branded SLB Works New 1.5mm T...
2        Product name is  Generic 1 Pc brand snail eye ...
3        Product name is  Generic Anti Snoring Snore St...
4        Product name is  Harveys Crunchy & Creame Gour...
                               ...                        
27383    Product name is  New Day Creams & Moisturizers...
27384    Product name is  Dermasolve Psoriasis Body Was...
27385    Product name is  Pure Organics Skin Lightening...
27386    Product name is  Yashus Moisturizing Anti-agin...
27387    Product name is  Slss Anti Wrinkle Organic Hya...
Name: title_description, Length: 27388, dtype: object

In [6]:
df["description_vector"] = df["title_description"].apply(lambda x: model.encode(x))

KeyboardInterrupt: 

: 

In [6]:
df.to_csv('./final.csv', index=0)

In [19]:
record_list = df.to_dict("records")

In [41]:
for record in record_list:
    try:
        record['description_vector'] = 'null'
        client.index(index="ecommerce_search", document=record, id=record["id"], timeout='100ms')
    except Exception as e:
        print(e)
    break

ApiError(503, 'unavailable_shards_exception', '[ecommerce_search][0] primary shard is not active Timeout: s], request: [BulkShardRequest [[ecommerce_search][0]] containing [index {[ecommerce_search][eb49cc038190f6f03c272f79fbbce894], source[{"id":"eb49cc038190f6f03c272f79fbbce894","category":"Skin Care","title":" Lee posh Lactic Acid 60% Anti ageing Pigmentation Removing Glow Peel ","description":"PROFESSIONAL GRADE Face Peel: this peel stimulates collagen production, reducing the appearance of wrinkles, fine lines, and hyper pigmentation in the skin by increasing cell regeneration Highly effective professional strength superficial solution Read direction for use on bottle of product & if any query call customer care","brand":"Lee Posh","mrp":"2000.00","price":"799.00","offers":"60.05%","stock_availibility":true,"product_asin":"B072BGHNJ1","image_urls":"https://images-na.ssl-images-amazon.com/images/I/41l0RjF1XIL._SS40_.jpg|https://images-na.ssl-images-amazon.com/images/I/41kBVM4KPwL._

In [40]:
client.count(index='ecommerce_search')

ApiError: ApiError(503, 'search_phase_execution_exception', None)

In [23]:
df['category'].value_counts()

Skin Care                  13656
Grocery & Gourmet Foods     7369
Hair Care                   2261
Fragrance                   1966
Bath & Shower               1960
Detergents & Dishwash        176
Name: category, dtype: int64

In [5]:

input_keyword = "Lee posh"
vector_of_input_keyword = model.encode(input_keyword)

query = {
    "field" : "description_vector",
    "query_vector" : vector_of_input_keyword,
    "k" : 5,
    "num_candidates" : 500, 
}

res = client.knn_search(index="ecommerce_search", knn=query, source=["title","description", 'category'], 
    filter= {
    "match": {
      "category": {
        "query": "Skin Care",
        "boost": 0.9
      }
    }
  })
hits = res["hits"]["hits"]
print(len(hits))
if not hits:
    print("No matches found")
else:
    for hit in hits:
        score = hit["_score"]
        product = hit["_source"]["title"]
        category = hit["_source"]["category"]
        description = hit["_source"]["description"]
        print(
            f"\nScore: {score}\nProduct:{product}\nCategory: {category}\nDescription: {description}\n"
        )

NotFoundError: NotFoundError(404, 'index_not_found_exception', 'no such index [ecommerce_search]', ecommerce_search, index_or_alias)

## Only elasticseach

In [13]:
res = client.search(
    index='ecommerce', 
    query={
        "match": {
            "product": {
                "query": "iRobot",
                # "analyzer": "stop",
            }
        }
    },
    fields=['product', 'description', 'category'])
hits = res["hits"]["hits"]

if not hits:
    print("No matches found")
else:
    for hit in hits:
        score = hit["_score"]
        product = hit["_source"]["product"]
        category = hit["_source"]["category"]
        description = hit["_source"]["description"]
        print(
            f"\nScore: {score}\nProduct: {product}\nCategory: {category}\nDescription: {description}\n"
        )


Score: 7.9402328
Product: iRobot Vacuum Cleaner
Category: Robot Vacuum Cleaner
Description: is an intelligent robotic vacuum that automatically cleans your floors. It uses advanced sensors and mapping technology to efficiently navigate and clean various surfaces.


Score: 6.6676292
Product: iRobot Roomba i7+ Robot Vacuum
Category: Robot Vacuum Cleaners
Description: is an advanced robot vacuum with self-emptying capability. It uses smart mapping for efficient cleaning and smart home integration.


Score: 6.6676292
Product: iRobot Roomba i4+ Robot Vacuum
Category: Robot Vacuum Cleaners
Description: is a smart robot vacuum with self-emptying capability. It offers advanced navigation and personalized cleaning schedules.

